# Implementación de ejemplos SimPy

In [27]:
'''
Librerias base
'''

import random
import simpy
import collections
import itertools

# Ejemplo 1 (Bank Renege - Renegación en el Banco)

**Cubiertas:**

*Recursos:* Recurso
Eventos de condicion
Un contador con un tiempo de servicio aleatorio y clientes que renuncian. Basado en el programa bank08.py del tutorial TheBank de SimPy 2. (KGM)

Este ejemplo modela un contador bancario y los clientes que llegan t veces al azar. Cada cliente tiene una cierta paciencia. Espera para llegar al mostrador hasta que termina su atadura. Si llega al mostrador, lo usa durante un tiempo antes de soltarlo.

El proceso source crea nuevos clientes cada pocos pasos.

In [28]:
'''
Cubiertas:

- Recursos: Recurso
- Condición de los eventos.

Guión:
   Un contador con un tiempo de servicio aleatorio y clientes que renuncian. Basado en el
   programa bank08.py de TheBank tutorial de SimPy 2. (KGM)
'''

RANDOM_SEED = 42 #Semilla para el generador aleatorio
NEW_CUSTOMERS = 5  # Numero total de clientes
INTERVAL_CUSTOMERS = 10.0  # Generador de clientes aproximadamente cada x segundos
MIN_PATIENCE = 1  # Minimo de la paciencia del cliente
MAX_PATIENCE = 3  # Maximo de la paciencia del cliente


def source(env, number, interval, counter):
    """Fuente generadora de clientes al azar"""
    for i in range(number):
        c = customer(env, 'Cliente #%02d' % i, counter, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


def customer(env, name, counter, time_in_bank):
    """El cliente llega, es atendido y se va."""
    arrive = env.now
    print('%7.4f %s: Estoy aquí' % (arrive, name))

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Espere el contador o aborte al final de su cuerda.
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            # Llegamos al mostrador
            print('%7.4f %s: Esperando %6.3f' % (env.now, name, wait))

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print('%7.4f %s: Finalizó' % (env.now, name))

        else:
            # Renegamos
            print('%7.4f %s: Desesperado después de %6.3f' % (env.now, name, wait))


# Configura y comienza la simulación.
print('Impaciencia en el banco')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Iniciar procesos y ejecutar
counter = simpy.Resource(env, capacity=1)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run()

Impaciencia en el banco
 0.0000 Cliente #00: Estoy aquí
 0.0000 Cliente #00: Esperando  0.000
 3.8595 Cliente #00: Finalizó
10.2006 Cliente #01: Estoy aquí
10.2006 Cliente #01: Esperando  0.000
12.7265 Cliente #02: Estoy aquí
13.9003 Cliente #02: Desesperado después de  1.174
23.7507 Cliente #01: Finalizó
34.9993 Cliente #03: Estoy aquí
34.9993 Cliente #03: Esperando  0.000
37.9599 Cliente #03: Finalizó
40.4798 Cliente #04: Estoy aquí
40.4798 Cliente #04: Esperando  0.000
43.1401 Cliente #04: Finalizó


# Ejemplo 2 (Carwash - Lavado de autos)

**Cubiertas:**

Esperando otros procesos

*Recursos:* Recurso
El Carwash ejemplo es una simulación de un tren de lavado con un número limitado de máquinas y un número de vehículos que llegan al tren de lavado para ser limpiado.

El lavado de autos usa una Resourcepara modelar el número limitado de lavadoras. También define un proceso para lavar un automóvil.

Cuando un automóvil llega al lavado de autos, solicita una máquina. Una vez que obtuvo uno, comienza el proceso de lavado del lavado de autos y espera a que termine. Finalmente libera la máquina y se va.

Los coches son generados por un proceso de configuración . Después de crear una cantidad inicial de autos, crea nuevos procesos de autos luego de un intervalo de tiempo aleatorio, siempre y cuando la simulación continúe.

In [29]:
"""
Cubiertas:

- Esperando otros procesos.
- Recursos: Recurso

Guión:
   Un lavado de autos tiene un número limitado de lavadoras y define
   Un proceso de lavado que lleva algún tiempo (aleatorio).

   Los procesos del automóvil llegan al lavado de autos en un tiempo aleatorio. Si uno de lavado
   La máquina está disponible, inician el proceso de lavado y la esperan.
   para terminar. Si no, esperan hasta que usen uno.

"""


RANDOM_SEED = 42
NUM_MACHINES = 2  # Número de máquinas en el lavado de autos.

WASHTIME = 5 # Minutos para limpiar un auto
T_INTER = 7 # Crea un auto cada ~ 7 minutos
SIM_TIME = 20 # Tiempo de simulación en minutos


class Carwash(object):
    """Un lavado de autos tiene un número limitado de máquinas (`` NUM_MACHINES``) para
     coches limpios en paralelo.

     Los coches tienen que solicitar una de las máquinas. Cuando consiguieron uno,
     puede iniciar el proceso de lavado y esperar a que termine (lo que
     toma `` washtime`` minutos).

    """
    def __init__(self, env, num_machines, washtime):
        self.env = env
        self.machine = simpy.Resource(env, num_machines)
        self.washtime = washtime

    def wash(self, car):
        """Los procesos de lavado. Toma un `` car`` procesa e intenta
         para limpiarlo."""
        yield self.env.timeout(WASHTIME)
        print("Lavado de autos eliminado %d%% de %s's la suciedad de." %
              (random.randint(50, 99), car))


def car(env, name, cw):
    """El proceso del automóvil (cada automóvil tiene un `` nombre``) llega al lavado de autos
     (`` cw``) y solicita una máquina de limpieza.

     Luego comienza el proceso de lavado, espera a que termine y
     deja para nunca volver ...

    """
    print('%s llega al lavado de autos en %.2f.' % (name, env.now))
    with cw.machine.request() as request:
        yield request

        print('%s entra en el lavado de autos en %.2f.' % (name, env.now))
        yield env.process(cw.wash(name))

        print('%s deja el lavado de autos en %.2f.' % (name, env.now))


def setup(env, num_machines, washtime, t_inter):
    """Crea un lavado de autos, una serie de autos iniciales y sigue creando autos
     aprox. cada `` t_inter`` minutos."""
    # Crea el lavado de autos
    carwash = Carwash(env, num_machines, washtime)

    # Crea 4 autos iniciales.
    for i in range(4):
        env.process(car(env, 'Carro %d' % i, carwash))

    # Crea más coches mientras se ejecuta la simulación.
    while True:
        yield env.timeout(random.randint(t_inter - 2, t_inter + 2))
        i += 1
        env.process(car(env, 'Carro %d' % i, carwash))


# Configura y comienza la simulación.
print('Lavado de autos')
print('Echa un vistazo a http://youtu.be/fXXmeP9TvBg mientras simula ... ;-)')
random.seed(RANDOM_SEED)  # Esto ayuda a reproducir los resultados.

# Crea un entorno y comienza el proceso de configuración.
env = simpy.Environment()
env.process(setup(env, NUM_MACHINES, WASHTIME, T_INTER))

# ¡Ejecutar!
env.run(until=SIM_TIME)

Lavado de autos
Echa un vistazo a http://youtu.be/fXXmeP9TvBg mientras simula ... ;-)
Carro 0 llega al lavado de autos en 0.00.
Carro 1 llega al lavado de autos en 0.00.
Carro 2 llega al lavado de autos en 0.00.
Carro 3 llega al lavado de autos en 0.00.
Carro 0 entra en el lavado de autos en 0.00.
Carro 1 entra en el lavado de autos en 0.00.
Carro 4 llega al lavado de autos en 5.00.
Lavado de autos eliminado 97% de Carro 0's la suciedad de.
Lavado de autos eliminado 67% de Carro 1's la suciedad de.
Carro 0 deja el lavado de autos en 5.00.
Carro 1 deja el lavado de autos en 5.00.
Carro 2 entra en el lavado de autos en 5.00.
Carro 3 entra en el lavado de autos en 5.00.
Carro 5 llega al lavado de autos en 10.00.
Lavado de autos eliminado 64% de Carro 2's la suciedad de.
Lavado de autos eliminado 58% de Carro 3's la suciedad de.
Carro 2 deja el lavado de autos en 10.00.
Carro 3 deja el lavado de autos en 10.00.
Carro 4 entra en el lavado de autos en 10.00.
Carro 5 entra en el lavado de aut

# Ejemplo 3 (Machine Shop - Tienda de máquinas)

**Cubiertas:**

Interrupciones

*Recursos:* PreemptiveResource

Este ejemplo comprende un taller con n máquinas idénticas. Llega una serie de trabajos (suficientes para mantener las máquinas ocupadas). Cada máquina se descompone periódicamente. Las reparaciones son realizadas por un reparador. El reparador también tiene otras tareas menos importantes que realizar. Las máquinas rotas se adelantan a estas tareas. El reparador los continúa cuando termina con la reparación de la máquina. El taller funciona continuamente.

Una máquina tiene dos procesos: trabajar implementa el comportamiento real de la máquina (producir partes). break_machine interrumpe periódicamente el proceso de trabajo para simular la falla de la máquina.

El otro trabajo del reparador también es un proceso (implementado por other_job ). El reparador en sí es un PreemptiveResource con una capacidad de 1 . La reparación de la máquina tiene una prioridad de 1 , mientras que el otro trabajo tiene una prioridad de 2 (cuanto menor sea el número, mayor será la prioridad).

In [30]:
"""
Cubiertas:

- Interrupciones
- Recursos: PreemptiveResource

Guión:
   Un taller tiene * n * máquinas idénticas. Un flujo de trabajos (suficiente para
   mantener las máquinas ocupadas) llega. Cada máquina se descompone
   periódicamente Las reparaciones son realizadas por un reparador. El reparador
   También tiene otras tareas menos importantes que realizar. Máquinas rotas
   Tareas previas a las tesis. El reparador los continúa cuando termina.
   Con la reparación de la máquina. El taller funciona continuamente.
"""

RANDOM_SEED = 42
PT_MEAN = 10.0 # Promedio tiempo de procesamiento en minutos
PT_SIGMA = 2.0 # Sigma del tiempo de procesamiento
MTTF = 300.0 # Tiempo promedio hasta la falla en minutos
BREAK_MEAN = 1 / MTTF # Param. para la distribucion expovariable
REPAIR_TIME = 30.0 # Tiempo que se tarda en reparar una máquina en minutos
JOB_DURATION = 30.0 # Duración de otros trabajos en minutos
NUM_MACHINES = 10 # Número de máquinas en el taller de máquinas
WEEKS = 4 # Tiempo de simulación en semanas
SIM_TIME = WEEKS * 7 * 24 * 60 # Tiempo de simulación en minutos


def time_per_part():
    """Devolver el tiempo de procesamiento real para una parte concreta."""
    return random.normalvariate(PT_MEAN, PT_SIGMA)


def time_to_failure():
    """Tiempo de vuelta hasta el siguiente fallo de una máquina."""
    return random.expovariate(BREAK_MEAN)


class Machine(object):
    """Una máquina produce partes y me rompen de vez en cuando.

     Si se rompe, solicita un * reparador * y continúa la producción.
     después de que se repare.

     Una máquina tiene un * nombre * y un número de * partes_hechas * hasta ahora.

    """
    def __init__(self, env, name, repairman):
        self.env = env
        self.name = name
        self.parts_made = 0
        self.broken = False

        # Comience con los procesos "trabajando" y "rompiendo la máquina" para esta máquina.
        self.process = env.process(self.working(repairman))
        env.process(self.break_machine())

    def working(self, repairman):
        """Produce partes siempre que la simulación se ejecute.

         Al hacer una parte, la máquina puede romperse varias veces.
         Solicite un reparador cuando esto suceda.

        """
        while True:
            # Empezar a hacer una nueva parte
            done_in = time_per_part()
            while done_in:
                try:
                    # Trabajando en la parte
                    start = self.env.now
                    yield self.env.timeout(done_in)
                    done_in = 0  # Establézcalo en 0 para salir mientras bucle.

                except simpy.Interrupt:
                    self.broken = True
                    done_in -= self.env.now - start  # ¿Cuanto tiempo falta?

                    # Solicitar un reparador. Esto anulará su "other_job".
                    with repairman.request(priority=1) as req:
                        yield req
                        yield self.env.timeout(REPAIR_TIME)

                    self.broken = False

            #La parte esta hecha.
            self.parts_made += 1

    def break_machine(self):
        """Romper la máquina de vez en cuando."""
        while True:
            yield self.env.timeout(time_to_failure())
            if not self.broken:
                # Sólo rompa la máquina si está trabajando actualmente.
                self.process.interrupt()


def other_jobs(env, repairman):
    """El otro trabajo (sin importancia) del reparador.."""
    while True:
        # Iniciar un nuevo trabajo
        done_in = JOB_DURATION
        while done_in:
            # Vuelva a intentar el trabajo hasta que esté hecho.
            # Su prioridad es menor que la de reparaciones de máquinas.
            with repairman.request(priority=2) as req:
                yield req
                try:
                    start = env.now
                    yield env.timeout(done_in)
                    done_in = 0
                except simpy.Interrupt:
                    done_in -= env.now - start


# Configura y comienza la simulación.
print('Tienda de máquina')
random.seed(RANDOM_SEED)  # Esto ayuda a reproducir los resultados.

# Crea un entorno y comienza el proceso de configuración.
env = simpy.Environment()
repairman = simpy.PreemptiveResource(env, capacity=1)
machines = [Machine(env, 'Máquina %d' % i, repairman)
            for i in range(NUM_MACHINES)]
env.process(other_jobs(env, repairman))

# ¡Ejecutar!
env.run(until=SIM_TIME)

# Resultados de analisis
print('Resultados del taller de la máquina después de %s semanas' % WEEKS)
for machine in machines:
    print('%s hecho %d partes.' % (machine.name, machine.parts_made))

Tienda de máquina
Resultados del taller de la máquina después de 4 semanas
Máquina 0 hecho 3251 partes.
Máquina 1 hecho 3273 partes.
Máquina 2 hecho 3242 partes.
Máquina 3 hecho 3343 partes.
Máquina 4 hecho 3387 partes.
Máquina 5 hecho 3244 partes.
Máquina 6 hecho 3269 partes.
Máquina 7 hecho 3185 partes.
Máquina 8 hecho 3302 partes.
Máquina 9 hecho 3279 partes.


# Ejemplo 4 (Movie Renege - Renegación en la Pelicula)

**Cubiertas:**

*Recursos:* Recurso
Eventos de condicion
Eventos compartidos

Este ejemplo modela una sala de cine con un mostrador de boletos que vende boletos para tres películas (solo la próxima presentación). La gente llega en momentos aleatorios y trata de comprar un número aleatorio (1–6) para una película al azar. Cuando una película está agotada, todas las personas que esperan comprar un boleto para esa cancelación de película (dejar la cola).

El cine es solo un contenedor para todos los datos relacionados (películas, el contador, las entradas que quedan, los datos recopilados, ...). El contador es un Resourcecon una capacidad de uno.

El espectador proceso se inicia en espera hasta que es su turno (que adquiere el recurso contador) o hasta que la vendió la señal se activa. Si este último es el caso, renuncia (deja la cola). Si llega al mostrador, trata de comprar algunas entradas. Esto podría no tener éxito, por ejemplo, si el proceso intenta comprar 5 boletos pero solo quedan 3. Si quedan menos de dos boletos después de la compra del boleto, se activa la señal de agotado .

Los espectadores son generados por el proceso de llegada de clientes . También elige una película y el número de entradas para el espectador.

In [31]:
"""
Cubiertas:

- Recursos: Recurso
- Condición de los eventos.
- Eventos compartidos

Guión:
   Un cine tiene un mostrador de boletos que vende boletos para tres.
   películas (solo show). Cuando una película se agota, todas las personas esperan.
   para comprar entradas para esa película renegar (dejar cola).

"""

RANDOM_SEED = 42
TICKETS = 50 # Número de entradas por película
SIM_TIME = 120 # Simular hasta


def moviegoer(env, movie, num_tickets, theater):
    """Un espectador lo intenta por una cantidad de tickets (* num_tickets *) para
     una cierta * película * en un * teatro *.

     Si la película se agota, ella abandona el teatro. Si ella consigue
     Al mostrador, ella trata de comprar un número de entradas. Si no es suficiente
     Quedan entradas, ella discute con el cajero y se va.

     Si a lo sumo queda un boleto después de que el espectador la compró
     entradas, el evento * agotado * para esta película se activa causando
     todos los espectadores restantes para salir.

    """
    with theater.counter.request() as my_turn:
        # Espera hasta que sea nuestro turno o hasta que la película se agote.
        result = yield my_turn | theater.sold_out[movie]

        # Compruebe si es nuestro turno de si la película está agotada
        if my_turn not in result:
            theater.num_renegers[movie] += 1
            env.exit()

        # Compruebe si quedan suficientes entradas.
        if theater.available[movie] < num_tickets:
            # El espectador se va después de una discusión
            yield env.timeout(0.5)
            env.exit()

        # Comprar boletos
        theater.available[movie] -= num_tickets
        if theater.available[movie] < 2:
            # Activa el evento "agotado" para la película.
            theater.sold_out[movie].succeed()
            theater.when_sold_out[movie] = env.now
            theater.available[movie] = 0
        yield env.timeout(1)


def customer_arrivals(env, theater):
    """Crear nuevos * espectadores * hasta que el tiempo de simulación llegue a 120."""
    while True:
        yield env.timeout(random.expovariate(1 / 0.5))

        movie = random.choice(theater.movies)
        num_tickets = random.randint(1, 6)
        if theater.available[movie]:
            env.process(moviegoer(env, movie, num_tickets, theater))


Theater = collections.namedtuple('Theater', 'counter, movies, available, '
                                            'sold_out, when_sold_out, '
                                            'num_renegers')

# Configura y comienza la simulación.
print('Renegue de la pelicula')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Crear cine
counter = simpy.Resource(env, capacity=1)
movies = ['Magia Salvaje', 'Mr Robot', 'Los Simpsons']
available = {movie: TICKETS for movie in movies}
sold_out = {movie: env.event() for movie in movies}
when_sold_out = {movie: None for movie in movies}
num_renegers = {movie: 0 for movie in movies}
theater = Theater(counter, movies, available, sold_out, when_sold_out,
                  num_renegers)

# Iniciar proceso y ejecutar
env.process(customer_arrivals(env, theater))
env.run(until=SIM_TIME)

# Resultados de analisis
for movie in movies:
    if theater.sold_out[movie]:
        print('La película "%s" se agotó %.1f minutos después del contador de boletos apertura.' % (movie, theater.when_sold_out[movie]))
        print(' Número de personas que salen de la fila cuando la película se agotó: %s' %
              theater.num_renegers[movie])

Renegue de la pelicula
La película "Magia Salvaje" se agotó 38.0 minutos después del contador de boletos apertura.
 Número de personas que salen de la fila cuando la película se agotó: 16
La película "Mr Robot" se agotó 43.0 minutos después del contador de boletos apertura.
 Número de personas que salen de la fila cuando la película se agotó: 5
La película "Los Simpsons" se agotó 28.0 minutos después del contador de boletos apertura.
 Número de personas que salen de la fila cuando la película se agotó: 5


# Ejemplo 5 (Gas Station Refueling - Estación de Gasolina)

**Cubiertas:**

*Recursos:* 

Recursos: Recurso
Recursos: Contenedor

Esperando otros procesos
Este ejemplo modela una estación de servicio y autos que llegan a la estación para repostar.

La estación de servicio tiene un número limitado de bombas de combustible y un tanque de combustible compartido entre las bombas de combustible. La gasolinera se modela así como Resource. El tanque de combustible compartido está modelado con un Container.

Los vehículos que llegan a la estación de servicio solicitan primero una bomba de combustible de la estación. Una vez que adquieren uno, intentan tomar la cantidad deseada de combustible de la bomba de combustible. Se van cuando terminan.

El nivel de las estaciones de gas combustible se controla periódicamente por control de la estación de gas . Cuando el nivel cae por debajo de un cierto umbral, se llama a un camión cisterna para repostar la estación de servicio.

In [32]:
"""
Cubiertas:

- Recursos: Recurso
- Recursos: Contenedor
- Esperando otros procesos.

Guión:
   Una estación de servicio tiene un número limitado de bombas de gas que comparten una
   depósito de combustible. Los autos llegan al azar a la estación de servicio, solicite uno
   de las bombas de combustible y comenzar a repostar desde ese depósito.

   Un proceso de control de la estación de gas observa el nivel de combustible de la estación de gas.
   y llama a un camión cisterna para repostar si baja el nivel de la estación
   por debajo de un umbral.

"""

RANDOM_SEED = 42
GAS_STATION_SIZE = 200 # litros
THRESHOLD = 10 # Umbral para llamar al camión cisterna (en%)
FUEL_TANK_SIZE = 50 # litros
FUEL_TANK_LEVEL = [5, 25] # Niveles mínimos / máximos de tanques de combustible (en litros)
REFUELING_SPEED = 2 # litros / segundo
TANK_TRUCK_TIME = 300 # Segundos toma el camión tanque para llegar
T_INTER = [30, 300] # Crea un auto cada [min, max] segundos
SIM_TIME = 1000 # Tiempo de simulación en segundos

def car(name, env, gas_station, fuel_pump):
    """Llega un coche a la gasolinera para repostar.

     Solicita una de las bombas de combustible de la estación de servicio e intenta obtener el
     cantidad deseada de gas de ella. Si el embalse de las estaciones es
     agotado, el coche tiene que esperar a que llegue el camión cisterna.

    """
    fuel_tank_level = random.randint(*FUEL_TANK_LEVEL)
    print('%s llegando a la gasolinera en %.1f' % (name, env.now))
    with gas_station.request() as req:
        start = env.now
        # Solicitar una de las bombas de gas.
        yield req

        # Solicitar una de las bombas de gas....
        liters_required = FUEL_TANK_SIZE - fuel_tank_level
        yield fuel_pump.get(liters_required)

        # El proceso de repostaje "real" lleva algún tiempo
        yield env.timeout(liters_required / REFUELING_SPEED)

        print('%s terminado de repostar en %.1f segundos.' % (name,
                                                          env.now - start))


def gas_station_control(env, fuel_pump):
    """Verifique periódicamente el nivel de la * fuel_pump * y llame al tanque
     camión si el nivel cae por debajo de un umbral."""
    while True:
        if fuel_pump.level / fuel_pump.capacity * 100 < THRESHOLD:
            # ¡Necesitamos llamar al camión cisterna ahora!
            print('Llamando camión cisterna al %d' % env.now)
            # Espera a que llegue el camión cisterna y repostar la estación.
            yield env.process(tank_truck(env, fuel_pump))

        yield env.timeout(10)  # Compruebe cada 10 segundos


def tank_truck(env, fuel_pump):
    """Llega a la estación de servicio después de un cierto retraso y lo reabastece.."""
    yield env.timeout(TANK_TRUCK_TIME)
    print('Camión cisterna llegando a la hora %d' % env.now)
    ammount = fuel_pump.capacity - fuel_pump.level
    print('Camión tanque de reabastecimiento de combustible %.1f litros.' % ammount)
    yield fuel_pump.put(ammount)


def car_generator(env, gas_station, fuel_pump):
    """Generar autos nuevos que llegan a la estación de servicio.."""
    for i in itertools.count():
        yield env.timeout(random.randint(*T_INTER))
        env.process(car('Carro %d' % i, env, gas_station, fuel_pump))


# Configura y comienza la simulación.
print('Estación de gasolina o Gasolinería')
random.seed(RANDOM_SEED)

# Crear entorno e iniciar procesos.
env = simpy.Environment()
gas_station = simpy.Resource(env, 2)
fuel_pump = simpy.Container(env, GAS_STATION_SIZE, init=GAS_STATION_SIZE)
env.process(gas_station_control(env, fuel_pump))
env.process(car_generator(env, gas_station, fuel_pump))

# ¡Ejecutar!
env.run(until=SIM_TIME)

Estación de gasolina o Gasolinería
Carro 0 llegando a la gasolinera en 87.0
Carro 0 terminado de repostar en 18.5 segundos.
Carro 1 llegando a la gasolinera en 129.0
Carro 1 terminado de repostar en 19.0 segundos.
Carro 2 llegando a la gasolinera en 284.0
Carro 2 terminado de repostar en 21.0 segundos.
Carro 3 llegando a la gasolinera en 385.0
Carro 3 terminado de repostar en 13.5 segundos.
Carro 4 llegando a la gasolinera en 459.0
Llamando camión cisterna al 460
Carro 4 terminado de repostar en 22.0 segundos.
Carro 5 llegando a la gasolinera en 705.0
Carro 6 llegando a la gasolinera en 750.0
Camión cisterna llegando a la hora 760
Camión tanque de reabastecimiento de combustible 188.0 litros.
Carro 6 terminado de repostar en 29.0 segundos.
Carro 5 terminado de repostar en 76.5 segundos.
Carro 7 llegando a la gasolinera en 891.0
Carro 7 terminado de repostar en 13.0 segundos.


# Ejemplo 6 (Process Communication - Proceso de comunicación)

**Cubiertas:**

*Recursos:*  Tienda

Este ejemplo muestra cómo interconectar elementos del modelo de simulación usando "recursos.Almacenar" para procesos asíncronos uno a uno y muchos a uno. Para uno a muchos, una simple clase BroadCastPipe se construye desde la tienda.

*Cuando sea útil:*

Cuando un proceso de consumidor no siempre espera en un proceso de generación y estos procesos se ejecutan de forma asíncrona. Este ejemplo muestra cómo crear un búfer y también saber si el proceso del consumidor se retrasó en el evento debido a un proceso de generación.

Esto también es útil cuando es necesario transmitir cierta información a muchos procesos de recepción.

Finalmente, el uso de tuberías puede simplificar cómo los procesos se interconectan entre sí en un modelo de simulación.

Ejemplo por:
Keith Smith

In [33]:
"""

Cubiertas:

- Recursos: Tienda

Guión:
   Este ejemplo muestra cómo interconectar elementos del modelo de simulación.
   juntos usando: class: `~ simpy.resources.store.Store` para uno a uno,
   y procesos asíncronos muchos a uno. Para uno a muchos un simple
   La clase BroadCastPipe se construye desde la tienda.

Cuando sea útil:
   Cuando un proceso de consumo no siempre espera en un proceso de generación.
   y estos procesos se ejecutan de forma asíncrona. Este ejemplo muestra cómo
   crear un búfer y también decir es que el proceso del consumidor se retrasó
   cediendo al evento desde un proceso generador.

   Esto también es útil cuando se necesita transmitir cierta información a
   muchos procesos de recepción

   Finalmente, el uso de tuberías puede simplificar cómo se interconectan los procesos
   entre sí en un modelo de simulación.

Ejemplo por:
   Keith Smith

"""

RANDOM_SEED = 42
SIM_TIME = 100


class BroadcastPipe(object):
    """Una canalización de difusión que permite que un proceso envíe mensajes a muchos.

     Esta construcción es útil cuando los consumidores de mensajes se están ejecutando en
     Diferentes tarifas que los generadores de mensajes y proporciona un evento.
     buffering a los procesos consumidores.

     Los parámetros se utilizan para crear una nueva.
     : class: `~ instancia simpy.resources.store.Store` cada vez
     : Se llama a: meth: `get_output_conn ()`.

    """
    def __init__(self, env, capacity=simpy.core.Infinity):
        self.env = env
        self.capacity = capacity
        self.pipes = []

    def put(self, value):
        """Transmitir un * valor * a todos los receptores."""
        if not self.pipes:
            raise RuntimeError('No hay tuberías de salida..')
        events = [store.put(value) for store in self.pipes]
        return self.env.all_of(events)  # Condición de evento para todos los "eventos".

    def get_output_conn(self):
        """Obtenga una nueva conexión de salida para esta tubería de difusión.

         El valor de retorno es a: class: `~ simpy.resources.store.Store`.

        """
        pipe = simpy.Store(self.env, capacity=self.capacity)
        self.pipes.append(pipe)
        return pipe


def message_generator(name, env, out_pipe):
    """Un proceso que genera mensajes al azar.."""
    while True:
        # espera la próxima transmisión
        yield env.timeout(random.randint(6, 10))

        # los mensajes tienen una marca de tiempo para verificar más tarde si el consumidor estaba
        # tarde conseguirlos. Tenga en cuenta que usar event.triggered para hacer esto puede
        # resultado en falla debido a la naturaleza FIFO de los rendimientos de simulación.
        # (es decir, si en el mismo env.now, message_generator pone un mensaje
        # en la tubería primero y luego message_consumer obtiene de la tubería,
        # el evento. activado será True en el otro orden que será
        # Falso
        
        msg = (env.now, '%s dice hola a %d' % (name, env.now))
        out_pipe.put(msg)


def message_consumer(name, env, in_pipe):
    """Un proceso que consume mensajes.."""
    while True:
        # Obtener evento para canalización de mensajes
        msg = yield in_pipe.get()

        if msg[0] < env.now:
            # si el mensaje ya fue puesto en la tubería, entonces
            # message_consumer llegó tarde a eso. Dependiendo de qué
            # está siendo modelado esto, puede, o puede no tener algunos
            # significado
            print('TARDE Obteniendo el mensaje: en el momento% d: %s mensaje recibido:  %s' % (env.now, name, msg[1]))
        else:
            # message_consumer está sincronizado con message_generator
            print('en el momento %d: %s mensaje recibido: %s.' %
                  (env.now, name, msg[1]))

        # El proceso realiza algún otro trabajo, lo que puede provocar que falten mensajes.
        yield env.timeout(random.randint(4, 8))


# Configura y comienza la simulación.
print('Proceso de comunicacion')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Para tuberías de uno a uno o de muchos a uno, use Store
pipe = simpy.Store(env)
env.process(message_generator('Generador A', env, pipe))
env.process(message_consumer('Consumidor A', env, pipe))

print('\nComunicación unipersonal (uno a uno)\n')
env.run(until=SIM_TIME)

# Para uno a muchos uso BroadcastPipe
# (Nota: también se puede utilizar para uno a uno, muchos a uno o muchos a muchos)
env = simpy.Environment()
bc_pipe = BroadcastPipe(env)

env.process(message_generator('Generador A', env, bc_pipe))
env.process(message_consumer('Consumidor A', env, bc_pipe.get_output_conn()))
env.process(message_consumer('Consumidor B', env, bc_pipe.get_output_conn()))

print('\nComunicación de uno a muchos por tubería\n')
env.run(until=SIM_TIME)

Proceso de comunicacion

Comunicación unipersonal (uno a uno)

en el momento 6: Consumidor A mensaje recibido: Generador A dice hola a 6.
en el momento 12: Consumidor A mensaje recibido: Generador A dice hola a 12.
en el momento 19: Consumidor A mensaje recibido: Generador A dice hola a 19.
en el momento 26: Consumidor A mensaje recibido: Generador A dice hola a 26.
en el momento 36: Consumidor A mensaje recibido: Generador A dice hola a 36.
en el momento 46: Consumidor A mensaje recibido: Generador A dice hola a 46.
en el momento 52: Consumidor A mensaje recibido: Generador A dice hola a 52.
en el momento 58: Consumidor A mensaje recibido: Generador A dice hola a 58.
TARDE Obteniendo el mensaje: en el momento 66: Consumidor A mensaje recibido:  Generador A dice hola a 65
en el momento 75: Consumidor A mensaje recibido: Generador A dice hola a 75.
en el momento 85: Consumidor A mensaje recibido: Generador A dice hola a 85.
en el momento 95: Consumidor A mensaje recibido: Generador A di

# Ejemplo 7 (Event Latency - Latencia del evento)

**Cubiertas:**

*Recursos:*  Tienda

Este ejemplo muestra cómo separar el tiempo de espera de los eventos entre los procesos de los procesos en sí.

Cuando sea útil:
Al modelar cosas físicas como cables, propagación de RF, etc., es mejor encapsular para mantener este mecanismo de propagación fuera de los procesos de envío y recepción.

También se puede utilizar para interconectar procesos enviando mensajes.

Ejemplo por:
Keith Smith

In [34]:
"""
Cubiertas:

- Recursos: Tienda

Guión:
   Este ejemplo muestra cómo separar el tiempo de espera de los eventos entre
   Procesos a partir de los procesos mismos.

Cuando sea útil:
   Al modelar cosas físicas como cables, propagación de RF, etc.
   Mejor encapsulación para mantener este mecanismo de propagación fuera de la
   Procesos de envío y recepción.

   También se puede utilizar para interconectar procesos enviando mensajes.

Ejemplo por:
   Keith Smith

"""

SIM_DURATION = 100


class Cable(object):
    """Esta clase representa la propagación a través de un cable.."""
    def __init__(self, env, delay):
        self.env = env
        self.delay = delay
        self.store = simpy.Store(env)

    def latency(self, value):
        yield self.env.timeout(self.delay)
        self.store.put(value)

    def put(self, value):
        self.env.process(self.latency(value))

    def get(self):
        return self.store.get()


def sender(env, cable):
    """Un proceso que genera mensajes al azar.."""
    while True:
        # espera la próxima transmisión
        yield env.timeout(5)
        cable.put('El remitente envió esto a %d' % env.now)


def receiver(env, cable):
    """Un proceso que consume mensajes.."""
    while True:
        # Obtener evento para canalización de mensajes
        msg = yield cable.get()
        print('Recibí esto en %d mientras %s' % (env.now, msg))


# Configura y comienza la simulación.
print('Latencia del evento')
env = simpy.Environment()

cable = Cable(env, 10)
env.process(sender(env, cable))
env.process(receiver(env, cable))

env.run(until=SIM_DURATION)

Latencia del evento
Recibí esto en 15 mientras El remitente envió esto a 5
Recibí esto en 20 mientras El remitente envió esto a 10
Recibí esto en 25 mientras El remitente envió esto a 15
Recibí esto en 30 mientras El remitente envió esto a 20
Recibí esto en 35 mientras El remitente envió esto a 25
Recibí esto en 40 mientras El remitente envió esto a 30
Recibí esto en 45 mientras El remitente envió esto a 35
Recibí esto en 50 mientras El remitente envió esto a 40
Recibí esto en 55 mientras El remitente envió esto a 45
Recibí esto en 60 mientras El remitente envió esto a 50
Recibí esto en 65 mientras El remitente envió esto a 55
Recibí esto en 70 mientras El remitente envió esto a 60
Recibí esto en 75 mientras El remitente envió esto a 65
Recibí esto en 80 mientras El remitente envió esto a 70
Recibí esto en 85 mientras El remitente envió esto a 75
Recibí esto en 90 mientras El remitente envió esto a 80
Recibí esto en 95 mientras El remitente envió esto a 85
